In [1]:
import numpy as np
import pandas as pd
import nltk

In [2]:
df = pd.read_csv('/content/tweets.csv')

In [3]:
df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [4]:
pd.set_option('display.max_colwidth',None)
df = df[['label','tweet']]
df.columns = ['label','tweet']
df.head()

,label,tweet
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


#preprocessing
#Gensim

In [5]:
#pip install --upgrade gensim

In [7]:
import gensim
df['text_clean_gensim'] = df['tweet'].apply(lambda x: gensim.utils.simple_preprocess(x))
df.head()

,label,tweet,text_clean_gensim
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,"[fingerprint, pregnancy, test, https, goo, gl, mfqv, android, apps, beautiful, cute, health, igers, iphoneonly, iphonesia, iphone]"
1,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/,"[finally, transparant, silicon, case, thanks, to, my, uncle, yay, sony, xperia, sonyexperias, http, instagram, com, yget, jc, jm]"
2,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu,"[we, love, this, would, you, go, talk, makememories, unplug, relax, iphone, smartphone, wifi, connect, http, fb, me, lsupcu]"
3,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/,"[wired, know, george, was, made, that, way, iphone, cute, daventry, home, http, instagr, am, li_]"
4,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!,"[what, amazing, service, apple, won, even, talk, to, me, about, question, have, unless, pay, them, for, their, stupid, support]"


In [10]:
x =df['text_clean_gensim']
y = df['label']

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

#Training W2Vec

In [12]:
w2v_model = gensim.models.Word2Vec(X_train,min_count=1)

In [29]:
w2v_model.wv['hey']

array([-0.07292072,  0.20991805,  0.16190094,  0.1550614 ,  0.06946042,
       -0.4654553 ,  0.18680936,  0.610273  , -0.3036226 , -0.26615706,
       -0.13467363, -0.4654355 ,  0.00502756,  0.16134435,  0.09687269,
       -0.10312438,  0.14056693, -0.23489828, -0.06637409, -0.65061384,
        0.21435624,  0.00354761,  0.3039893 , -0.31340885, -0.04860612,
        0.0135856 , -0.22452176, -0.16747247, -0.21813123, -0.0510128 ,
        0.21641321,  0.12715863,  0.17137925, -0.38400587, -0.14598058,
        0.33221787,  0.1764053 , -0.24591129, -0.24761473, -0.45687252,
        0.15842472, -0.24733315, -0.10748938,  0.04114027,  0.24201582,
       -0.2128438 , -0.2809143 , -0.11022191,  0.22540154,  0.3042748 ,
        0.20635208, -0.19191292, -0.10554612, -0.14002763, -0.06911013,
        0.05061187,  0.09092365,  0.10164738, -0.26659477,  0.02933631,
        0.02727412, -0.00369514,  0.05547104,  0.1081719 , -0.26703548,
        0.3432985 ,  0.06790075,  0.22055763, -0.37133196,  0.33

In [30]:
len(w2v_model.wv['hey'])

100

#vectorization


In [31]:
w2v_model.wv.most_similar('hey')

[('he', 0.9989041090011597),
 ('guys', 0.9986817836761475),
 ('into', 0.9986373782157898),
 ('over', 0.9985320568084717),
 ('customer', 0.9984736442565918),
 ('days', 0.9984148144721985),
 ('steve', 0.9984000325202942),
 ('after', 0.9983712434768677),
 ('man', 0.9983526468276978),
 ('waiting', 0.9983381032943726)]

In [16]:
words = w2v_model.wv.index_to_key

In [17]:
len(words)

19072

In [18]:
X_train_vect =[]
X_test_vect = []
for ls in X_train:
  vector=[]
  for word in ls:
    if word in words:
      vector.append(w2v_model.wv[word])
  X_train_vect.append(np.array(vector))

In [19]:
X_test_vect = []
for ls in X_test:
  vectors=[]
  for word in ls:
    if word in words:
      vectors.append(w2v_model.wv[word])
  X_test_vect.append(np.array(vectors))

In [21]:
X_train_vect_avg = []
for v in X_train_vect:
  if v.size:
    X_train_vect_avg.append(v.mean(axis=0))
  else:
    X_train_vect_avg.append(np.zeros(100,dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
  if v.size:
    X_test_vect_avg.append(v.mean(axis=0))
  else:
    X_test_vect_avg.append(np.zeros(100,dtype=float))


#training the model

In [22]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train_vect_avg,y_train)

RandomForestClassifier()

In [32]:
from sklearn import metrics
y_pred = clf.predict(X_train_vect_avg)
metrics.accuracy_score(y_pred,y_train)

0.9998421717171717

#prediction

In [72]:
sms = "you are bad"
sms = gensim.utils.simple_preprocess(sms)

In [73]:
X_test1 = np.array([w2v_model.wv[i] for i in sms if i in words ]).mean(axis=0)

In [77]:
y_pred = clf.predict([X_test1])
y_pred

array([1])